In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained emotion detection model (replace with the path to your model)
emotion_model = tf.keras.models.load_model('emotiondetector.h5')

# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the labels for emotion classes
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Define a dictionary to map emotion labels to corresponding colors
emotion_colors = {
    'Angry': (0, 0, 255),      # Red
    'Disgust': (0, 255, 0),    # Green
    'Fear': (255, 0, 0),       # Blue
    'Happy': (0, 255, 255),    # Yellow
    'Sad': (128, 0, 128),      # Purple
    'Surprise': (255, 255, 0), # Cyan
    'Neutral': (128, 128, 128) # Gray
}

# Initialize video capture (you can specify 0 for the default camera)
cap = cv2.VideoCapture('Downloads\\emo1.mp4')

# Define the output video file name and codec
output_file = 'output_emotion_detection.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can try different codecs like 'XVID' or 'MP4V'

# Get the original frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the desired output size (you can change these values)
output_width = 640
output_height = 480

# Create a VideoWriter object to save the output video
out = cv2.VideoWriter(output_file, fourcc, 30, (output_width, output_height))

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the frame to the desired output size
    frame = cv2.resize(frame, (output_width, output_height))

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest (ROI) for emotion detection
        face_roi = gray[y:y + h, x:x + w]

        # Resize the ROI to match the input size of the emotion detection model
        face_roi = cv2.resize(face_roi, (48, 48))

        # Normalize the ROI
        face_roi = face_roi / 255.0

        # Reshape the ROI for prediction
        face_roi = face_roi.reshape(1, 48, 48, 1)

        # Perform emotion prediction
        emotion_prediction = emotion_model.predict(face_roi)
        emotion_index = np.argmax(emotion_prediction)
        emotion = emotion_labels[emotion_index]

        # Display the emotion label near the face
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, emotion_colors[emotion], 2)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture, output video, and close the OpenCV window
cap.release()
out.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 35ms/step
